In [8]:
import pandas as pd
from matplotlib import pyplot as plt
import os

In [11]:
path = "../output/kmeans/"
df_each = []
for filename in os.listdir(path):
    f = os.path.join(path, filename)
    if os.path.isfile(f):
        temp = pd.read_csv(f)
        temp = temp[temp.name.apply(lambda x: x != "name")]
        df_each.append((temp, filename))

In [12]:
print(len(df_each))
df_each[0][0].columns

1


Index(['name', 'run_num', 'params', 'reduction_time', 'accuracy',
       'original_std_sum', 'original_std_mean', 'original_std_median',
       'original_std_max', 'original_std_min', 'original_sparsity',
       'transformed_std_sum', 'transformed_std_mean', 'transformed_std_median',
       'transformed_std_max', 'transformed_std_min', 'transformed_sparsity',
       'characteristics'],
      dtype='object')

In [14]:
df_each[0][0].groupby(by=["name", "params"])["reduction_time"].min(), df_each[0][1]

(name                           params                  
 JL transform                   {'ep': 0.05, 'de': 0.05}    0.009709618
                                {'ep': 0.05, 'de': 0.1}     0.008342635
                                {'ep': 0.1, 'de': 0.05}     0.008689303
                                {'ep': 0.1, 'de': 0.1}      0.012512708
                                {'ep': 0.5, 'de': 0.05}     0.011248207
                                {'ep': 0.5, 'de': 0.1}      0.021972683
                                {'ep': 0.9, 'de': 0.05}     0.007082351
                                {'ep': 0.9, 'de': 0.1}      0.013333077
 Nothing                        {}                          0.001645217
 PCA                            {}                          0.001803286
 extremely sparse JL transform  {'ep': 0.05, 'de': 0.05}    0.018596254
                                {'ep': 0.05, 'de': 0.1}     0.010561676
                                {'ep': 0.1, 'de': 0.05}      0.01083787
       

In [4]:
def solution_1(n):
    nums = [i for i in range(1, n)]
    dp = [0 for _ in range(n + 1)]
    dp[0] = 1
    current_sum = 0
    for i in range(len(nums)):
        current_sum += nums[i]
        mn = min(current_sum, n)
        for j in range(mn, nums[i] - 1, -1):
            dp[j] += dp[j - nums[i]]
    return dp[n]

print(solution_1(10))

9


In [3]:
solution_1(200)

487067745

In [11]:
def sol_2(n):
    cnt = 0
    while n > 1:
        if n == 3:
            cnt += 2
            break
        elif n % 2 == 0:
            n /= 2
        elif n % 4 == 1:
            n -= 1
        else:
            n += 1
        cnt += 1
    return cnt

[(i, sol_2(i)) for i in range(2, 100)]

[(2, 1),
 (3, 2),
 (4, 2),
 (5, 3),
 (6, 3),
 (7, 4),
 (8, 3),
 (9, 4),
 (10, 4),
 (11, 5),
 (12, 4),
 (13, 5),
 (14, 5),
 (15, 5),
 (16, 4),
 (17, 5),
 (18, 5),
 (19, 6),
 (20, 5),
 (21, 6),
 (22, 6),
 (23, 6),
 (24, 5),
 (25, 6),
 (26, 6),
 (27, 7),
 (28, 6),
 (29, 7),
 (30, 6),
 (31, 6),
 (32, 5),
 (33, 6),
 (34, 6),
 (35, 7),
 (36, 6),
 (37, 7),
 (38, 7),
 (39, 7),
 (40, 6),
 (41, 7),
 (42, 7),
 (43, 8),
 (44, 7),
 (45, 8),
 (46, 7),
 (47, 7),
 (48, 6),
 (49, 7),
 (50, 7),
 (51, 8),
 (52, 7),
 (53, 8),
 (54, 8),
 (55, 8),
 (56, 7),
 (57, 8),
 (58, 8),
 (59, 8),
 (60, 7),
 (61, 8),
 (62, 7),
 (63, 7),
 (64, 6),
 (65, 7),
 (66, 7),
 (67, 8),
 (68, 7),
 (69, 8),
 (70, 8),
 (71, 8),
 (72, 7),
 (73, 8),
 (74, 8),
 (75, 9),
 (76, 8),
 (77, 9),
 (78, 8),
 (79, 8),
 (80, 7),
 (81, 8),
 (82, 8),
 (83, 9),
 (84, 8),
 (85, 9),
 (86, 9),
 (87, 9),
 (88, 8),
 (89, 9),
 (90, 9),
 (91, 9),
 (92, 8),
 (93, 9),
 (94, 8),
 (95, 8),
 (96, 7),
 (97, 8),
 (98, 8),
 (99, 9)]

In [20]:
sol_2(10000000000)

40

In [23]:
mp = {}

def minops(n, steps):
    if n == 1: return steps
    if n in mp: return mp[n] + steps
    if n % 2 == 0:
        cnt = minops(n/2, steps + 1)
        mp[n] = cnt - steps
        return cnt
    cnt = min(minops(n - 1, steps + 1), minops(n + 1, steps + 1))
    mp[n] = cnt - steps
    return cnt

minops(10000000000, 0)

40